<a href="https://colab.research.google.com/github/kamrulhasan04947/mlprac/blob/main/SentementAnalysisUsingLSTM%26_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
import numpy as np

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import os

In [ ]:
def dataBuilder(path):
    nameOflabels = ['neg', 'pos']
    if os.path.exists(path):
        sentiment = []
        label = []
        for dirname in os.listdir(path):
            if dirname in nameOflabels:
                class_name = nameOflabels.index(dirname)
                datapath = os.path.join(path,dirname)
                for file in os.listdir(datapath):
                    fl = open(os.path.join(datapath, file), 'r')
                    sentiment.append(fl.read())
                    label.append(class_name)
    return pd.DataFrame({'sentiments': sentiment, 'label': label})

In [ ]:
train_data = dataBuilder('/content/drive/MyDrive/dataset/data/train')
test_data = dataBuilder('/content/drive/MyDrive/dataset/data/test')

In [ ]:
train_data

,sentiments,label
0,"An unfunny, unworthy picture which is an undes...",0
1,"This movie really, i mean REALLY, sucks. Its g...",0
2,This flick reminds me some really bad science-...,0
3,Many people like to point to this TV movie whe...,0
4,"Please, spare me of these movies that teach us...",0
...,...,...
24995,"This is surprisingly above average slasher, th...",1
24996,By the late forties the era of the screwball c...,1
24997,What can I say about it?It's another Hollywood...,1
24998,How anyone can say this is bad is beyond me. I...,1


In [ ]:
test_data

,sentiments,label
0,Just saw it at as closing film of Austin Film ...,1
1,I watched two very different Holmes adventures...,1
2,King Vladislav (Angus Scrimm) of Romania is a ...,1
3,"In all truth, this really isn't a ""movie"" so m...",1
4,If you haven't seen this movie than you need t...,1
...,...,...
24998,Awful in a bad/good way...this movie has offic...,0
24999,This movie was painfully awful. Most of the mo...,0
25000,This movie stunk. There is not much more to it...,0
25001,"Uhhh ... so, did they even have writers for th...",0


In [ ]:
test_data.drop(test_data.tail(3).index, inplace=True)

In [ ]:
test_data

,sentiments,label
0,Just saw it at as closing film of Austin Film ...,1
1,I watched two very different Holmes adventures...,1
2,King Vladislav (Angus Scrimm) of Romania is a ...,1
3,"In all truth, this really isn't a ""movie"" so m...",1
4,If you haven't seen this movie than you need t...,1
...,...,...
24995,"Calling this a ""Sunday School"" movie might be ...",0
24996,So tell me - what serious boozer drinks Budwei...,0
24997,"There are movies that are leaders, and movies ...",0
24998,Awful in a bad/good way...this movie has offic...,0


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *

In [ ]:
wordnet = WordNetLemmatizer()

def pcontext_processor(context):
  #tokenization applying
  # find the sentences from paragraph
  corpus = [] # creating an empty list
  for i in range(0, len(context)): # iterating through sentences
    sent_data = nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', context['sentiments'][i]).lower())# removing punction and numbers extept words that have letters (a-z and A-Z ) # converting all words lower
    sent_data = [wordnet.lemmatize(word) for  word  in sent_data if word not in set(stopwords.words('english'))] # iterating through list of words to remove stop words and lamatized thest word by applying list comprehansion.
    sent_data = ' '.join(sent_data) # gathering words in a itaretive form
    corpus.append(sent_data) # appending sentence on corpus list
  return corpus

In [ ]:
train_courpus = pcontext_processor(train_data)

In [ ]:
test_courpus = pcontext_processor(test_data)

In [ ]:
test_courpus

['saw closing film austin film festival going seen snippet act alamo drafthouse thought trailer amusing expecting epic theme music use tarot card genius title sequence watch cameo ben stiller tim robbins dark lord fetish kg much top fun song catchy car chase far favorite scene know earn cult status right away win new tenacious fan like hope well theater follow shadow',
 'watched two different holmes adventure morning would amazed similarity br br presumably first collaboration basil rathbone king romeo juliet nigel bruce director roy william neill first sherlock holmes adventure one couple year br br made excellent team prefer hammer version talk later br br holmes relies lot disguise work constantly amused mannerism displayed figure something always seem dash find new clue br br story keeping bomb site german wwii interesting kept focused mystery',
 'king vladislav angus scrimm romania vampire vampire light want nothing live peace harmony mankind son radu anders hove cruel creature he

In [ ]:
vocab_size = 10000  # Adjustable based on your vocabulary size
max_sequence_length = 20  # Adjustable based on your desired sequence length
embedding_dim = 100
lstm_units = 64
num_classes = 3  # Number of classes (positive, negative, neutral)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, datasets, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_courpus)
tokenizer.fit_on_texts(test_courpus)
train_sequence = tokenizer.texts_to_sequences(train_courpus)
test_sequence = tokenizer.texts_to_sequences(test_courpus)
padded_train_sequences = pad_sequences(train_sequence, maxlen=max_sequence_length, padding="post", truncating="post")
padded_test_sequences = pad_sequences(train_sequence, maxlen=max_sequence_length, padding="post", truncating="post")


In [ ]:
y_train = pd.get_dummies(train_data['label'])
y_test = pd.get_dummies(test_data['label'])

In [ ]:
y_train= y_train.iloc[:, 1].values
y_test = y_test.iloc[:, 1].values

In [ ]:
categorical_train_labels = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
categorical_test_labels = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=(64, )),
    layers.MaxPool1D(5),
    LSTM(lstm_units),
    tf.keras.layers.Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(padded_train_sequences, categorical_train_labels, epochs=10, batch_size=32, validation_data=(padded_test_sequences, categorical_test_labels))


Epoch 1/10
782/782 [==============================] - 24s 26ms/step - loss: 0.5858 - accuracy: 0.6820 - val_loss: 1.5236 - val_accuracy: 0.1650
Epoch 2/10
782/782 [==============================] - 24s 30ms/step - loss: 0.3876 - accuracy: 0.8325 - val_loss: 2.4029 - val_accuracy: 0.0522
Epoch 3/10
782/782 [==============================] - 20s 25ms/step - loss: 0.1756 - accuracy: 0.9360 - val_loss: 4.6960 - val_accuracy: 0.0105
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.0448 - accuracy: 0.9856 - val_loss: 7.5720 - val_accuracy: 0.0027
Epoch 5/10
782/782 [==============================] - 20s 26ms/step - loss: 0.0187 - accuracy: 0.9938 - val_loss: 10.5158 - val_accuracy: 0.0014
Epoch 6/10
782/782 [==============================] - 23s 29ms/step - loss: 0.0130 - accuracy: 0.9959 - val_loss: 11.5303 - val_accuracy: 0.0018
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.0108 - accuracy: 0.9965 - val_loss: 13.1645 - val_accura

In [ ]:
score = model.evaluate(padded_test_sequences, categorical_test_labels, verbose = 0)

In [ ]:
print(score)

[14.269189834594727, 0.0012000000569969416]
